#   LangChain Tool 활용

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---

## **내장 도구 (Built-in Tool)**

- **내장 도구**는 시스템에 **기본적으로 포함**된 사전 정의된 도구들의 집합

- 파일 처리, 웹 검색, 코드 실행 등 **자주 사용되는 기본 기능**들을 즉시 활용 가능

- 별도의 구현 없이 **바로 사용**할 수 있어 개발 시간과 노력을 절약 가능

---

### **웹 검색(Tavily Search API)**

- **Tavily**는 AI 에이전트(LLM)를 위해 특별히 설계된 검색 엔진

- 검색 결과로 **제목, URL, 컨텐츠, 답변** 등 상세 정보 제공

- 개발자는 **월 1,000회** 무료 검색 쿼터 사용 가능

- **설치**: `pip install tavily-python` 또는 `uv add tavily-python`

- **인증키**: `TAVILY_API_KEY`

`(1) 도구 정의`

In [4]:
from langchain_community.tools import TavilySearchResults

search_web = TavilySearchResults(
    max_results=5,  # 반환할 결과의 수
    search_depth="advanced",  # 검색 깊이: basic 또는 advanced
    include_answer=True,  # 결과에 직접적인 답변 포함
    include_raw_content=True,  # 페이지의 원시 콘텐츠 포함
    include_images=True,  # 결과에 이미지 포함
    # include_domains=[...],  # 특정 도메인으로 검색 제한
    # exclude_domains=[...],  # 특정 도메인 제외
    # name="...",  # 기본 도구 이름 덮어쓰기
    # description="...",  # 기본 도구 설명 덮어쓰기
    # args_schema=...,  # 기본 args_schema 덮어쓰기
)

C:\Users\User\AppData\Local\Temp\ipykernel_19192\1142060768.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_web = TavilySearchResults(


`(2) 도구 직접 호출`
- 자연어 쿼리를 도구에 전달

In [5]:
result = search_web.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
pprint(result)

[{'content': '# 국내 ETF 1000개 첫 돌파…최초 상장 뒤 23년만 성과\n'
             '\n'
             '아시아경제\n'
             '원문 \n'
             '기사전송 2025-07-22 22:00\n'
             '\n'
             '공유하기\n'
             '\n'
             '네이트온\n'
             '\n'
             '페이스북\n'
             '\n'
             '엑스\n'
             '\n'
             '기사 제목과 본문 내용을 자동 요약한 내용입니다.   \n'
             '전체 내용을 이해하기 위해서는 기사 본문 전체를 확인하는 것이 더 좋습니다.\n'
             '\n'
             '국내 상장지수펀드(ETF) 시장이 빠르게 성장하면서 증시에서 거래되는 ETF 수가 처음으로 1000개를 넘겼다.\n'
             '\n'
             '국내 ETF 1000개 첫 돌파…최초 상장 뒤 23년만 성과\n'
             '\n'
             '연합뉴스\n'
             '\n'
             '22일 한국거래소에 따르면 이날 기준 국내 시장에서 거래되고 있는 ETF는 1200개다. 2002년 국내 시장에 '
             'ETF가 처음 상장된 이후 23년 만에 1000개를 돌파한 것이다.\n'
             '\n'
             'ETF 수는 2023년 11월 21일 800개를 넘긴 뒤 지난해 10월 15일 900개를 웃도는 등 빠른 속도로 '
             '늘어왔다. [...] 이날도 더제이 중소형포커스액티브, KODEX TDF2060액티브, KIWOOM '
             '미국테크100월간목표헤지액티브 등 7

`(3) ToolCall을 사용한 호출`
- 모델에서 생성된 `ToolCall`을 사용하여 도구를 호출

In [6]:
from langchain_openai import ChatOpenAI

# 모델 생성
model = ChatOpenAI(model="gpt-4.1-mini")

# 모델에 도구 등록 
model_with_tools = model.bind_tools([search_web])

# 사용자 쿼리를 입력하여 ToolCall 생성
response = model_with_tools.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")

# ToolCall 출력 
pprint(response.tool_calls)

[{'args': {'query': '한국 시장 거래되는 ETF 종목 수'},
  'id': 'call_Vqb5cDpuYnMOxOZOEUiLisJn',
  'name': 'tavily_search_results_json',
  'type': 'tool_call'}]


In [7]:
# ToolCall을 사용하여 도구 실행
model_generated_tool_call = response.tool_calls[0]
tool_msg = search_web.invoke(model_generated_tool_call)

# 도구 실행 결과 출력
pprint(tool_msg.content)

('[{"title": "국내 ETF 현황 - Fund&ETFs - FnIndex", "url": '
 '"http://www2.fnspace.com:4000/fund/etf", "content": "국내 ETF 상장종목수| 구분 | '
 '2015.12 | 2016.12 | 2017.12 | 2018.12 | 2019.12 | 2020.12 | 2021.12 | '
 '2022.12 | 2023.12 | 2024.12 | 2025.05 |\\n| --- | --- | --- | --- | --- | '
 '--- | --- | --- | --- | --- | --- | --- |\\n| ETF(국내주식) | 펀드수 | 113 | 140 | '
 '176 | 234 | 257 | 256 | 291 | 316 | 348 | 367 | 372 |\\n| 비중 | 57.07% | '
 '54.69% | 54.15% | 56.66% | 57.11% | 54.70% | 54.60% | 47.45% | 42.80% | '
 '39.25% | 37.61% |\\n| ETF(해외주식) | 펀드수 | 40 | 55 | 70 | 76 | 74 | 88 | 116 | '
 '167 | 220 | 291 | 323 | [...] | 비중 | 20.20% | 21.48% | 21.54% | 18.40% | '
 '16.44% | 18.80% | 21.76% | 25.08% | 27.06% | 31.12% | 32.66% |\\n| ETF(국내채권) '
 '| 펀드수 | 21 | 28 | 37 | 50 | 53 | 54 | 53 | 74 | 106 | 129 | 132 |\\n| 비중 | '
 '10.61% | 10.94% | 11.38% | 12.11% | 11.78% | 11.54% | 9.94% | 11.11% | '
 '13.04% | 13.80% | 13.35% |\\n| ETF(기타) | 펀드수 | 24 | 33 | 42 | 53 | 66 | 70 | '
 '73 | 109

In [8]:
tool_msg

ToolMessage(content='[{"title": "국내 ETF 현황 - Fund&ETFs - FnIndex", "url": "http://www2.fnspace.com:4000/fund/etf", "content": "국내 ETF 상장종목수| 구분 | 2015.12 | 2016.12 | 2017.12 | 2018.12 | 2019.12 | 2020.12 | 2021.12 | 2022.12 | 2023.12 | 2024.12 | 2025.05 |\\n| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |\\n| ETF(국내주식) | 펀드수 | 113 | 140 | 176 | 234 | 257 | 256 | 291 | 316 | 348 | 367 | 372 |\\n| 비중 | 57.07% | 54.69% | 54.15% | 56.66% | 57.11% | 54.70% | 54.60% | 47.45% | 42.80% | 39.25% | 37.61% |\\n| ETF(해외주식) | 펀드수 | 40 | 55 | 70 | 76 | 74 | 88 | 116 | 167 | 220 | 291 | 323 | [...] | 비중 | 20.20% | 21.48% | 21.54% | 18.40% | 16.44% | 18.80% | 21.76% | 25.08% | 27.06% | 31.12% | 32.66% |\\n| ETF(국내채권) | 펀드수 | 21 | 28 | 37 | 50 | 53 | 54 | 53 | 74 | 106 | 129 | 132 |\\n| 비중 | 10.61% | 10.94% | 11.38% | 12.11% | 11.78% | 11.54% | 9.94% | 11.11% | 13.04% | 13.80% | 13.35% |\\n| ETF(기타) | 펀드수 | 24 | 33 | 42 | 53 | 66 | 70 | 73 | 109 | 139 | 148 | 162 |\\n| 비중 | 12.

In [9]:
# `artifact` 속성 출력
tool_msg.artifact

{'query': '한국 시장 거래되는 ETF 종목 수',
 'follow_up_questions': None,
 'answer': 'As of 2025, there are over 400 ETF products listed on the Korean market. The majority track domestic stocks. The number of ETFs has steadily increased over the years.',
 'images': ['https://img.hankyung.com/photo/202209/P312720220929000009360-010.png',
  'https://newsimg.sedaily.com/2022/11/07/26DJ92T9IN_1.png',
  'https://img.etoday.co.kr/pto_db/2022/10/20221017095722_1808059_1200_900.png',
  'https://wooatime.com/wp-content/uploads/2025/01/1-min-4-1021x1024.webp',
  'https://www.kcmi.re.kr/kcmifile/report_data/1776/card/cdatt_72825420240124110511.jpg'],
 'results': [{'url': 'http://www2.fnspace.com:4000/fund/etf',
   'title': '국내 ETF 현황 - Fund&ETFs - FnIndex',
   'content': '국내 ETF 상장종목수| 구분 | 2015.12 | 2016.12 | 2017.12 | 2018.12 | 2019.12 | 2020.12 | 2021.12 | 2022.12 | 2023.12 | 2024.12 | 2025.05 |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| ETF(국내주식) | 펀드수 | 113 | 140 | 17

In [10]:
pprint(tool_msg.artifact)

{'answer': 'As of 2025, there are over 400 ETF products listed on the Korean '
           'market. The majority track domestic stocks. The number of ETFs has '
           'steadily increased over the years.',
 'follow_up_questions': None,
 'images': ['https://img.hankyung.com/photo/202209/P312720220929000009360-010.png',
            'https://newsimg.sedaily.com/2022/11/07/26DJ92T9IN_1.png',
            'https://img.etoday.co.kr/pto_db/2022/10/20221017095722_1808059_1200_900.png',
            'https://wooatime.com/wp-content/uploads/2025/01/1-min-4-1021x1024.webp',
            'https://www.kcmi.re.kr/kcmifile/report_data/1776/card/cdatt_72825420240124110511.jpg'],
 'query': '한국 시장 거래되는 ETF 종목 수',
 'response_time': 4.24,
 'results': [{'content': '국내 ETF 상장종목수| 구분 | 2015.12 | 2016.12 | 2017.12 | '
                         '2018.12 | 2019.12 | 2020.12 | 2021.12 | 2022.12 | '
                         '2023.12 | 2024.12 | 2025.05 |\n'
                         '| --- | --- | --- | --- | --- |

`(4) LLM 체인과 연계`
- Tavily 도구를 언어 모델에 바인딩하고 사용자 입력을 처리하는 체인을 생성

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage

import datetime

# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 프롬프트 템플릿 정의
today = datetime.datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate(
    [
        ("system", f"당신은 도움이 되는 어시스턴트입니다. 오늘 날짜는 {today}입니다. 답변에 근거 또는 출처를 반드시 표시합니다."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# Tavily 도구를 모델에 바인딩
llm_with_tools = llm.bind_tools([search_web])

# 체인 생성
llm_chain = prompt | llm_with_tools

@chain  # 함수를 체인으로 사용하기 위해 데코레이터 추가
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    
    # 도구 호출 결과를 처리
    tool_msgs = []
    artifacts = []  # artifact를 저장할 리스트
    for tool_call in ai_msg.tool_calls:
        tool_result = search_web.invoke(tool_call)
        tool_msgs.append(ToolMessage(content=str(tool_result.content), tool_call_id=tool_call["id"]))
        artifacts.append(tool_result.artifact)  # artifact 저장
    
    # 최종 결과 반환
    final_response = llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)
    
    return final_response, artifacts

# 체인 호출
response, artifacts = tool_chain.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
print("===== 최종 응답 =====")
print(response.content)

===== 최종 응답 =====
한국 시장에서 거래되는 ETF는 2025년 6월 기준으로 약 984개에 달하며, 하반기에는 1000개를 돌파할 것으로 예상됩니다. (출처: 이데일리, 2025-06-05)

또한, 나무위키에 따르면 순자산 1000억원 이상인 중대형 ETF 상품만 해도 상당한 수가 있으며, 다양한 테마와 지수를 추종하는 ETF가 다수 상장되어 있습니다.

정확한 최신 ETF 수량은 한국거래소(KRX) 또는 금융 관련 데이터 서비스(예: FunETF, 한국예탁결제원 등)에서 실시간으로 확인하는 것이 권장됩니다. 

요약:  
- 2025년 6월 기준 ETF 종목 수: 약 984개  
- 향후 1000개 이상으로 증가 예상

참고 링크:  
- 이데일리 기사: https://marketin.edaily.co.kr/News/ReadE?newsId=02683046642198768  
- 나무위키 국내 ETF 목록: https://namu.wiki/w/%EA%B5%AD%EB%82%B4%20ETF%20%EB%AA%A9%EB%A1%9D  
- FunETF: https://www.funetf.co.kr/  
- 한국거래소 데이터: http://data.krx.co.kr/contents/MDC/EASY/visualController/MDCEASY400.cmd


In [ ]:
# artifact를 별도로 출력
print("===== Artifact 정보 =====")
for artifact in artifacts:
    print(json.dumps(artifact, indent=2, ensure_ascii=False))

---

##  **사용자 정의 도구 (Custom Tool)**


- **사용자 정의 도구**는 개발자가 직접 설계하고 구현하는 **맞춤형 함수나 도구**를 의미

- LLM이 호출할 수 있는 **고유한 기능**을 정의하여 특정 작업에 최적화된 도구 생성 가능

- 개발자는 도구의 **입력값, 출력값, 기능**을 자유롭게 정의하여 유연한 확장성 확보

---

### 1. **외부 API 연동** 

- LangChain에서는 **사용자 정의 도구**를 통해 외부 API와의 연동이 가능

- **Tool** 클래스를 상속받아 필요한 기능을 구현하며, `name`과 `description`을 필수로 정의

- 도구는 단일 기능을 수행하는 **함수 형태**로 구현되며, 입력과 출력이 명확하게 설정함

`(1) Yahoo Finance API` 

In [14]:
#  yfinance 설치 : pip install yfinance 또는 uv add yfinance
import yfinance as yf

dat = yf.Ticker("MSFT")

In [15]:
# 기업정보
dat.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [16]:
# 주요 일정
dat.calendar

{'Dividend Date': datetime.date(2025, 9, 11),
 'Ex-Dividend Date': datetime.date(2025, 8, 21),
 'Earnings Date': [datetime.date(2025, 7, 31)],
 'Earnings High': 3.57,
 'Earnings Low': 3.31,
 'Earnings Average': 3.37557,
 'Revenue High': 74464996940,
 'Revenue Low': 72570000000,
 'Revenue Average': 73795260110}

In [17]:
# 2022년 1월 3일 ~ 4일의 데이터를 판다스 데이터프레임으로 출력 
result = dat.history(start="2022-01-03", end="2022-01-05") 
result

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,325.620684,328.193796,320.212277,325.038086,28865100,0.0,0.0
2022-01-04 00:00:00-05:00,325.115662,325.474952,316.658371,319.464539,32674300,0.0,0.0


In [18]:
# 인덱스 초기화 
result = result.reset_index()
result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-01-03 00:00:00-05:00,325.620684,328.193796,320.212277,325.038086,28865100,0.0,0.0
1,2022-01-04 00:00:00-05:00,325.115662,325.474952,316.658371,319.464539,32674300,0.0,0.0


In [19]:
# 날짜 부분만 추출
result['Date'] = result['Date'].dt.strftime('%Y-%m-%d')

result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2022-01-03,325.620684,328.193796,320.212277,325.038086,28865100,0.0,0.0
1,2022-01-04,325.115662,325.474952,316.658371,319.464539,32674300,0.0,0.0


In [20]:
# 데이터프레임을 딕셔너리로 변환
result_dict = result.to_dict(orient='records') 
result_dict

[{'Date': '2022-01-03',
  'Open': 325.6206844003455,
  'High': 328.19379550970876,
  'Low': 320.2122765756348,
  'Close': 325.0380859375,
  'Volume': 28865100,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Date': '2022-01-04',
  'Open': 325.11566209587266,
  'High': 325.4749522851141,
  'Low': 316.6583710755657,
  'Close': 319.46453857421875,
  'Volume': 32674300,
  'Dividends': 0.0,
  'Stock Splits': 0.0}]

`(2) 데이터 연동 및 출력 포맷` 

In [ ]:
# 외부 API 연동하는 함수 (yfinance 사용)

from langchain_core.tools import ToolException
from typing import Dict, Optional
from datetime import datetime, timedelta
import yfinance as yf

def get_stock_price(symbol: str, date: Optional[str] = None) -> Dict:
    """yfiance 사용하여 특정 날짜의 주식의 가격 정보를 조회합니다."""

    if date and not is_valid_date(date):
        raise ToolException(f"잘못된 날짜 형식입니다: {date}")
    
    try:
        stock = yf.Ticker(symbol)
        # 특정 날짜의 주식 가격 정보 조회 
        if date:
            start = datetime.strptime(date, "%Y-%m-%d")
            end = start + timedelta(days=1)
            price = stock.history(start=start, end=end)

            # 가격 정보가 없으면 해날 날짜로부터 과거 5일간의 주식 가격 정보 조회
            if price.empty:
                end = start - timedelta(days=5)
                price = stock.history(start=end, end=start)

        # 특정 날짜가 없으면 최근 5일간의 주식 가격 정보 조회
        else:
            price = stock.history(period="5d")
            
        # 데이터프레임을 딕셔너리로 변환하여 반환 (가장 최근 날짜 데이터만 반환)
        df = price.reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df.to_dict(orient='records')[-1]
    
    except Exception as e:
        raise ToolException(str(e))
    

def is_valid_date(date_str: str) -> bool:
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False
    

# 함수 실행
result = get_stock_price("AAPL")

# 결과 출력
print(result)

{'Date': '2025-07-22', 'Open': 213.13999938964844, 'High': 214.9499969482422, 'Low': 212.22999572753906, 'Close': 214.39999389648438, 'Volume': 46300400, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [23]:
# 함수 실행 (날짜 지정)
result = get_stock_price("AAPL", "2025-01-01")
print(result)

$AAPL: possibly delisted; no price data found  (1d 2025-01-01 00:00:00 -> 2025-01-02 00:00:00)


{'Date': '2024-12-31', 'Open': 251.8325261089863, 'High': 252.670501064015, 'Low': 248.82975968702823, 'Close': 249.8173828125, 'Volume': 39480700, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(3) StructuredTool 도구 변환` 

In [24]:
from langchain_core.tools import StructuredTool

# StructuredTool로 도구 생성
stock_tool = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_basic",
    description="yfinance를 사용하여 주식 가격 정보를 조회하는 도구입니다.",
)

# 도구 실행 (정상)
result = stock_tool.invoke({"symbol": "AAPL"})
print(result)

{'Date': '2025-07-22', 'Open': 213.13999938964844, 'High': 214.9499969482422, 'Low': 212.22999572753906, 'Close': 214.39999389648438, 'Volume': 46300400, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(4) LLM 사용하여 도구 사용` 

In [25]:
from langchain_openai import ChatOpenAI

# OpenAI GPT-4o-mini 모델 사용
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 바인딩
llm_with_tool = llm.bind_tools([search_web, stock_tool])

# 도구 호출
result = llm_with_tool.invoke("애플의 주식 가격을 알려줘")
pprint(result.tool_calls)
print("-"*100)

# ToolCall을 도구에 전달하여 결과 확인
tool_call = result.tool_calls[0]
if tool_call['name'] == "stock_price_basic":
    print("도구 호출 이름:", tool_call['name'])
    result = stock_tool.invoke(tool_call)
elif tool_call['name'] == "tavily_search_results":
    print("도구 호출 이름:", tool_call['name'])
    result = search_web.invoke(tool_call)
else:
    raise ValueError(f"알 수 없는 도구 호출: {tool_call['name']}")

print(result)

[{'args': {'symbol': 'AAPL'},
  'id': 'call_n1leg0Ef1aVww78Cm6hktlPc',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
----------------------------------------------------------------------------------------------------
도구 호출 이름: stock_price_basic
content='{"Date": "2025-07-22", "Open": 213.13999938964844, "High": 214.9499969482422, "Low": 212.22999572753906, "Close": 214.39999389648438, "Volume": 46300400, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_n1leg0Ef1aVww78Cm6hktlPc'


In [26]:
# 도구 호출 (날짜 지정)
result = llm_with_tool.invoke("애플의 2025년 1월 3일 주식 가격을 알려줘")
pprint(result.tool_calls)

# ToolCall을 도구에 전달하여 결과 확인
for tool_call in result.tool_calls:
    print("도구 호출 이름:", tool_call['name'])
    if tool_call['name'] == "stock_price_basic":
        result = stock_tool.invoke(tool_call)
    elif tool_call['name'] == "tavily_search_results":
        result = search_web.invoke(tool_call)
    else:
        raise ValueError(f"알 수 없는 도구 호출: {tool_call['name']}")
    
    print(result)
    print("-" * 100)

[{'args': {'date': '2025-01-03', 'symbol': 'AAPL'},
  'id': 'call_ojMQPtYnFBOPSQDLg8pOvwEV',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
도구 호출 이름: stock_price_basic
content='{"Date": "2025-01-03", "Open": 242.7743682861328, "High": 243.5923870862029, "Low": 241.3079045417172, "Close": 242.7743682861328, "Volume": 40244100, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_ojMQPtYnFBOPSQDLg8pOvwEV'
----------------------------------------------------------------------------------------------------


In [27]:
from langgraph.prebuilt import create_react_agent

# 도구 실행 에인전트 생성 
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='8c25e54c-333c-4342-88f6-2d51ac0d5eab'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UmQ8awBjLWkHHxcqTmEnPLxR', 'function': {'arguments': '{"symbol":"AAPL","date":"2025-01-03"}', 'name': 'stock_price_basic'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BwJh5ecEGaIv7Yswj2YhnbtzDJ56r', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--462a8a42-9426-4450-8867-df3cbb48bf60-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': 'AAPL', 'dat

In [28]:
# 도구 실행 에인전트 생성 (return_direct=True)

stock_tool.return_direct = True
search_web.return_direct = True
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='d54186f1-ba62-4281-8b5a-c8f790cfa501'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3jvWC5NL1Q3BYP5kNz1gSZDE', 'function': {'arguments': '{"symbol":"AAPL","date":"2025-01-03"}', 'name': 'stock_price_basic'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BwJi6BKS2bGKy31iDTmf5TqlTZm8p', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--20a60fe8-0150-4d49-936b-fb0c9db35469-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': 'AAPL', 'dat

In [29]:
# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 최신 뉴스를 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 최신 뉴스를 알려줘', additional_kwargs={}, response_metadata={}, id='09276845-2bdd-4c8c-a549-030940573b03'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_treJOf7jkV0vlLVLFmHhfORz', 'function': {'arguments': '{"query":"Apple latest news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 129, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BwJiCjXX3UDqfMvmeHTr9mWfHzabr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--497521fd-cc39-4285-a8bd-ced9a75d96bb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Apple latest n

---

### 2. **Naver 개발자 API** 를 도구로 사용

- 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
- 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)
- 아래 정의된 네이버 뉴스 검색 도구(naver_news_search)를 사용하여 다음 과정을 수행합니다. 

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

`(1) 도구 정의`

In [30]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다."""

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {"query": query}

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

`(2) 도구 실행`

In [31]:
from langchain_core.runnables import chain, RunnableLambda
from langchain_openai import ChatOpenAI

# 도구 맵 생성 (도구 이름을 키로 사용)
tools = [stock_tool, naver_news_search]  
tool_map = {tool.name: tool for tool in tools}

# 도구 라우터 (도구 이름에 따라 실행할 도구를 선택)
@chain
def tool_router(tool_call):
    return tool_map[tool_call["name"]]

# 체인 구성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

tool_chain = (
    llm_with_tools    # 어떤 도구를 사용할지 결정 (LLM 모델이 도구 호출을 처리)
    | RunnableLambda(lambda x: x.tool_calls)  # 도구 호출을 추출
    | tool_router.map()   # 도구 호출 라우팅
)
# 도구 실행
response = tool_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

In [32]:
# 결과 출력 (도구 호출 결과)
for msg in response:
    pprint(msg)

ToolMessage(content='{"Date": "2025-07-22", "Open": 213.13999938964844, "High": 214.9499969482422, "Low": 212.22999572753906, "Close": 214.39999389648438, "Volume": 46300400, "Dividends": 0.0, "Stock Splits": 0.0}', name='stock_price_basic', tool_call_id='call_VajXHugX3reRITpz3hZ8ox9m')
ToolMessage(content='{"data": {"lastBuildDate": "Wed, 23 Jul 2025 13:10:27 +0900", "total": 89279, "start": 1, "display": 10, "items": [{"title": "미국 증시, <b>애플</b> 등 M7 쏠림 경고등", "originallink": "https://www.naeil.com/news/read/555645?ref=naver", "link": "https://www.naeil.com/news/read/555645?ref=naver", "description": "<b>최근</b> 2년간 투자자들은 이른바 ‘좋은’ 실적이 아닌 ‘경이적인’ 실적을 원해왔고, 기대에 미치지 못한 결과에는 매정하게 <b>주가</b>로 응답했다. 마이크로소프트의 지난해 <b>주가</b> 흐름을 보면 분기별 주당순이익(EPS)이... ", "pubDate": "Wed, 23 Jul 2025 13:02:00 +0900"}, {"title": "美증시 빅테크 천하…“러셀2000&lt;시총 하위 2000개 중소형주&gt;에 분산투자...", "originallink": "https://biz.heraldcorp.com/article/10537886?ref=naver", "link": "https://n.news.naver.com/mnews/article/016/0002503758

`(3) 체인 실행`

In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI
from datetime import datetime

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4.1-mini")

# 두 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[stock_tool, naver_news_search])

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다. 답변 생성의 근거 또는 출처를 명시하세요."),
    ("human", "{user_input}"),  
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def news_analysis_chain(user_input: str):

    # 도구 체인 실행 (LLM -> StockPrice/NaverNewsSearch)
    tool_msgs = tool_chain.invoke(user_input)

    print(f"Tool Messages: {tool_msgs}")

    # 도구 결과를 포맷팅
    formatted_messages = []
    for msg in tool_msgs:
        if isinstance(msg, ToolMessage) and len(msg.content) > 0:
            formatted_messages.append({"role": "assistant", "content": msg.content}) 

    print(f"Formatted Messages: {formatted_messages}")

    # 최종 응답 생성
    final_response = llm_chain.invoke({"user_input": user_input, "messages": formatted_messages})

    print(f"Final Response: {final_response}")

    return final_response

In [34]:
# 체인 실행
response = news_analysis_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

Tool Messages: [ToolMessage(content='{"Date": "2025-07-22", "Open": 213.13999938964844, "High": 214.9499969482422, "Low": 212.22999572753906, "Close": 214.39999389648438, "Volume": 46300400, "Dividends": 0.0, "Stock Splits": 0.0}', name='stock_price_basic', tool_call_id='call_3BzrUgnqG9Y2YuFW4TWvL7DY'), ToolMessage(content='{"data": {"lastBuildDate": "Wed, 23 Jul 2025 13:10:54 +0900", "total": 89279, "start": 1, "display": 10, "items": [{"title": "미국 증시, <b>애플</b> 등 M7 쏠림 경고등", "originallink": "https://www.naeil.com/news/read/555645?ref=naver", "link": "https://www.naeil.com/news/read/555645?ref=naver", "description": "<b>최근</b> 2년간 투자자들은 이른바 ‘좋은’ 실적이 아닌 ‘경이적인’ 실적을 원해왔고, 기대에 미치지 못한 결과에는 매정하게 <b>주가</b>로 응답했다. 마이크로소프트의 지난해 <b>주가</b> 흐름을 보면 분기별 주당순이익(EPS)이... ", "pubDate": "Wed, 23 Jul 2025 13:02:00 +0900"}, {"title": "美증시 빅테크 천하…“러셀2000&lt;시총 하위 2000개 중소형주&gt;에 분산투자...", "originallink": "https://biz.heraldcorp.com/article/10537886?ref=naver", "link": "https://n.news.naver.com/mnews/artic

In [35]:
# 최종 응답 출력 
print(response.content)

애플(AAPL)의 최근 주가는 2025년 7월 22일 기준으로 종가 214.40달러를 기록했습니다.

최근 애플 주가와 관련한 주요 뉴스 기사들을 요약해 드리면:
- 최근 2년간 투자자들은 경이적인 실적에 반응하며 애플 등 빅테크 기업 주가가 영향을 받고 있습니다.
- AI 랠리 등으로 빅테크 주가가 급격히 상승했지만, 중장기적으로는 조심스러운 전망도 있습니다.
- 7월 말 애플의 분기 실적 발표가 예정되어 있어 시장의 관심이 집중되고 있습니다.
- 일부에서는 관세 정책과 글로벌 생산 이슈 등이 애플 주가 상승의 제약 요소로 지적되고 있습니다.

자세한 뉴스 링크도 원하시면 알려주세요.
